In [3]:
addprocs(3)

3-element Array{Int64,1}:
 2
 3
 4

In [4]:
nprocs()

4

In [5]:
@everywhere include("../src/UNSflow.jl")
@everywhere using UNSflow


/Users/kiran/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/kiran/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/kiran/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/kiran/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the fo

In [6]:
cref = 1.
bref = 4.
sref = 4.

patch1 = patch(0., -2., 0., 0., "FlatPlate", 1., 0., 0.11, 10)
patch2 = patch(0., 0., 0., 0., "FlatPlate", 1., 0., 0.11, 10)
patch3 = patch(0., 2., 0., 0., "FlatPlate", 1., 0., 0.11, 5)
patchdata = [patch1; patch2; patch3]

alphadef = EldUpDef(45.*pi/180, pi/8, 0.8)
hdef = ConstDef(0.)
udef = ConstDef(1.)
kin = KinemDef3D(alphadef, hdef, udef)


surf = ThreeDSurf(cref, bref, sref, patchdata, kin, 1., 70, 35, 21)
field = ThreeDFlowField()
dtstar = 0.0075
nsteps =round(Int,2.5/dtstar) + 1

334

In [7]:
 mat = Array(Float64, 0, 4)

    mat = mat'
    
    surf2d = TwoDSurf[]
    field2d = TwoDFlowField[]
    kinem2d = KinemDef[]

    dt = dtstar*surf.cref/surf.uref

    t = 0.

    AR = surf.bref/surf.cref

    bc = zeros(surf.nspan)
    a03d = zeros(surf.nspan)
    cl = zeros(surf.nspan)
    cd = zeros(surf.nspan)
    cm = zeros(surf.nspan)

    lhs = zeros(surf.nspan, surf.nbterm)
    rhs = zeros(surf.nspan)
    bcoeff = zeros(surf.nbterm)

21-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [8]:
   if surf.kindef.vartype == "Constant"

        for i = 1:surf.nspan
            # Kinematics at all strips is the same
            
            push!(kinem2d, KinemDef(surf.kindef.alpha, surf.kindef.h, surf.kindef.u))
            push!(surf2d, TwoDSurf(surf.patchdata[1].coord_file, surf.patchdata[1].pvt, kinem2d[i], [surf.patchdata[1].lc;]))
            #If 3D flow field is defined with disturbances or external vortices, these should be transferred to the 2D flowfield
            push!(field2d, TwoDFlowField())
        end
    end

In [9]:
@time for i = 1:surf.nspan
            #Update kinematic parameters
            update_kinem(surf2d[i], t)
        
            #Update flow field parameters if any
            update_externalvel(field2d[i], t)

            #Update bound vortex positions
            update_boundpos(surf2d[i], dt)

            #Add a TEV with dummy strength
            place_tev(surf2d[i], field2d[i], dt)
        end
        

  0.190644 seconds (40.52 k allocations: 1.713 MB)


In [12]:
 kelv = KelvinConditionLLTldvm(surf, surf2d, field2d)
        
        #Solve for TEV strength to satisfy Kelvin condition
        
        soln = nlsolve(not_in_place(kelv), -0.01*ones(surf.nspan))
       

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in slice(::Array{Float64,2}, ::Vararg{Any,N}) at ./deprecated.jl:30
 in sumabs2j at /Users/kiran/.julia/v0.5/NLsolve/src/utils.jl:1 [inlined]
 in trust_region_(::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Float64, ::Bool) at /Users/kiran/.julia/v0.5/NLsolve/src/trust_region.jl:105
 in #nlsolve#17(::Symbol, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Function, ::Float64, ::Bool, ::NLsolve.#nlsolve, ::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}) at /Users/kiran/.julia/v0.5/NLsolve/src/nlsolve_func_defs.jl:24
 in nlsolve(::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}) at /Users/kiran/.julia/v0.5/NLsolve/src/nlsolve_func_defs.jl:13
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kiran/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloo

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01]
 * Zero: [-2.37688e-12,-3.92216e-12,-4.76738e-12,-5.20185e-12,-5.44482e-12,-5.63327e-12,-5.6965e-12,-5.76767e-12,-5.76326e-12,-5.78933e-12,-5.79396e-12,-5.81307e-12,-5.78811e-12,-5.74575e-12,-5.66453e-12,-5.6204e-12,-5.44773e-12,-5.17749e-12,-4.77433e-12,-3.90081e-12,-2.40133e-12]
 * Inf-norm of residuals: 0.000000
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 2
 * Jacobian Calls (df/dx): 2

In [41]:
for ispan = 1:surf.nspan
for i = 1:1500
        push!(field2d[ispan].lev,TwoDVort(1.0, 1.0, 0.00001, 0.02, 0.0, 0.0))
    end
end

In [22]:
surf.nspan

21

In [30]:
@time for i = 1:surf.nspan
            field2d[i].tev[length(field2d[i].tev)].s = soln.zero[i]

            #Update incduced velocities on airfoil
            update_indbound(surf2d[i], field2d[i])
            
            #Calculate downwash
            update_downwash(surf2d[i], [field2d[i].u[1],field2d[i].w[1]])
            
            #Calculate first two fourier coefficients
            update_a0anda1(surf2d[i])

            bc[i] = surf2d[i].a0[1] + 0.5*surf2d[i].aterm[1]
        end


  0.047860 seconds (443 allocations: 395.109 KB)


In [32]:
@time pmap((a1,a2)->update_indbound(a1,a2),surf2d,field2d)

  0.416952 seconds (1.37 M allocations: 29.247 MB, 1.44% gc time)


21-element Array{Any,1}:
 UNSflow.TwoDSurf(1.0,1.0,"FlatPlate",0.0,70,35,UNSflow.KinemDef(UNSflow.EldUpDef(0.785398,0.392699,0.8),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0)),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0455303,0.0910607,0.136591,0.182121,0.227652,0.273182,0.318712,0.364243,0.409773  …  2.73182,2.77735,2.82288,2.86841,2.91394,2.95947,3.005,3.05053,3.09606,3.14159],[0.0,0.000518163,0.00207158,0.00465703,0.00826915,0.0129005,0.0185414,0.0251802,0.0328031,0.0413943  …  0.958606,0.967197,0.97482,0.981459,0.9871,0.991731,0.995343,0.997928,0.999482,1.0],UNSflow.KinemPar(6.12399e-13,0.0,1.51114e-11,0.0,1.0,0.0),[-1.0075,-1.00698,-1.00543,-1.00284,-0.999231,-0.9946,-0.988959,-0.98232,-0.974697,-0.966106  …  -0.0488943,-0.0403031,-0.0326802,-0.0260414,-0.0204005,-0.0157691,-0.012157,-0.00957158,-0.00801816,-0.0075],[0.0,-3.76049e-16,-1.50342e-15,

In [43]:
@time for i = 1:surf.nspan
    wakeroll(surf2d[i], field2d[i], dt)
end


  2.652977 seconds (590 allocations: 5.081 MB, 0.24% gc time)


In [46]:
@time pmap((a1,a2,a3)->wakeroll(a1,a2,a3),surf2d,field2d, dt*ones(surf.nspan))

  1.974198 seconds (3.61 M allocations: 81.515 MB, 0.97% gc time)


21-element Array{Any,1}:
 UNSflow.TwoDFlowField(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),[0.0],[0.0],UNSflow.TwoDVort[UNSflow.TwoDVort(-0.00377676,2.68574e-5,-2.37688e-12,0.02,-0.00118917,0.00119367),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13)  …  UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.6193e-13,2.1234e-13),UNSflow.TwoDVort(1.0,1.0,0.0,0.02,-1.61